In [21]:
from __future__ import print_function

import os
import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate
import math


if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("/home/xilinx/IPBitFile/matrixmultiplication.bit")
    ipMatrixmul = ol.matrixmul_0

    N = 32
    M = 32
    P = 32

    A = np.zeros((N,M), dtype=int)
    B = np.zeros((N,M), dtype=int)
    AB = np.zeros((N,M), dtype=int)
    
    for i in range(N):
        for j in range (M):
            A[i][j] = i+j

    for i in range(M):
        for j in range(P):
            B[i][j] = i+j;

    for i in range(N):
        for j in range(M):
            ipMatrixmul.write(0x1000 + i*128 + j*4, int(A[i][j]))
    for i in range(N):
        for j in range(M):
            ipMatrixmul.write(0x2000 + i*128 + j*4, int(B[i][j]))        

    print("running Design Under Test\n")
    ipMatrixmul.write(0x00, 0x01)
    
    for i in range(N):
        for j in range(M):
            AB[i][j]=ipMatrixmul.read(0x3000 + i*128 + j*4)

    # Print output
    fp = open("matrixmultiplication.out.dat", "w")
    print("Output:")
    for i in range(N):
        print(str(i)+"\t ", end='')
        for j in range(P):
            print(str(AB[i][j])+" ", end='')
            fp.write("{:>4d}".format(i))
            fp.write("\t")
            fp.write("{:>4d}".format(j))
            fp.write("\t")
            fp.write("{:d}".format(AB[i][j]))
            fp.write("\t")
            fp.write("\n")
        print("\n")
    
    print("Comparing against output data")
    if (os.system('diff -w matrixmultiplication.out.dat matrixmultiplication.gold.dat')):
        print("*******************************************")
        print("FAIL: Output DOES NOT match the golden output")
        print("*******************************************")
    else:
        print("*******************************************")
        print("PASS: The output matches the golden output!")
        print("*******************************************")
    


Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
running Design Under Test

Output:
0	 10416 10912 11408 11904 12400 12896 13392 13888 14384 14880 15376 15872 16368 16864 17360 17856 18352 18848 19344 19840 20336 20832 21328 21824 22320 22816 23312 23808 24304 24800 25296 25792 

1	 10912 11440 11968 12496 13024 13552 14080 14608 15136 15664 16192 16720 17248 17776 18304 18832 19360 19888 20416 20944 21472 22000 22528 23056 23584 24112 24640 25168 25696 26224 26752 27280 

2	 11408 11968 12528 13088 13648 14208 14768 15328 15888 16448 17008 17568 18128 18688 19248 19808 20368 20928 21488 22048 22608 23168 23728 24288 24848 25408 25968 26528 27088 27648 28208 28768 

3	 11904 12496 13088 13680 14272 14864 15456 16048 16640 17232 17824 18416 19008 19600 20192 20784 21376 21968 22560 23152 23744 24336 24928 25520 26112 26704 27296 27888 28480 29072 29664 30256 

4	 12400 13024 13648 14272 1489